<a href="https://colab.research.google.com/github/katia-kitaguti/ab-testing-analysis/blob/main/202406_%7C_AB_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries & Settings

In [27]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from google.colab import userdata
import os

from scipy.stats import shapiro
from scipy.stats import ttest_ind

In [28]:
os.environ["My_Key"] = userdata.get('My_Key')
os.environ["User_Name"] = userdata.get('User_Name')

In [29]:
!kaggle datasets download -d amirmotefaker/ab-testing-dataset

Dataset URL: https://www.kaggle.com/datasets/amirmotefaker/ab-testing-dataset
License(s): other
ab-testing-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [30]:
! unzip "ab-testing-dataset.zip"

Archive:  ab-testing-dataset.zip
replace control_group.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: control_group.csv       
replace test_group.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: test_group.csv          


# 1. Base & Data Treatments

### Control Base

In [31]:
df_control = pd.read_csv('control_group.csv', sep=';')
df_control

,Campaign Name,Date,Spend [USD],# of Impressions,Reach,# of Website Clicks,# of Searches,# of View Content,# of Add to Cart,# of Purchase
0,Control Campaign,1.08.2019,2280,82702.0,56930.0,7016.0,2290.0,2159.0,1819.0,618.0
1,Control Campaign,2.08.2019,1757,121040.0,102513.0,8110.0,2033.0,1841.0,1219.0,511.0
2,Control Campaign,3.08.2019,2343,131711.0,110862.0,6508.0,1737.0,1549.0,1134.0,372.0
3,Control Campaign,4.08.2019,1940,72878.0,61235.0,3065.0,1042.0,982.0,1183.0,340.0
4,Control Campaign,5.08.2019,1835,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Control Campaign,6.08.2019,3083,109076.0,87998.0,4028.0,1709.0,1249.0,784.0,764.0
6,Control Campaign,7.08.2019,2544,142123.0,127852.0,2640.0,1388.0,1106.0,1166.0,499.0
7,Control Campaign,8.08.2019,1900,90939.0,65217.0,7260.0,3047.0,2746.0,930.0,462.0
8,Control Campaign,9.08.2019,2813,121332.0,94896.0,6198.0,2487.0,2179.0,645.0,501.0
9,Control Campaign,10.08.2019,2149,117624.0,91257.0,2277.0,2475.0,1984.0,1629.0,734.0


In [32]:
new_column_names = [
    'Campaign Name', 'Date', 'Amount Spent', 'Num Impressions', 'Reach',
    'Num Clicks', 'Num Searches', 'Num Views',
    'Num Added to cart', 'Num Purchases'
]

df_control.columns = new_column_names

df_control.head()

,Campaign Name,Date,Amount Spent,Num Impressions,Reach,Num Clicks,Num Searches,Num Views,Num Added to cart,Num Purchases
0,Control Campaign,1.08.2019,2280,82702.0,56930.0,7016.0,2290.0,2159.0,1819.0,618.0
1,Control Campaign,2.08.2019,1757,121040.0,102513.0,8110.0,2033.0,1841.0,1219.0,511.0
2,Control Campaign,3.08.2019,2343,131711.0,110862.0,6508.0,1737.0,1549.0,1134.0,372.0
3,Control Campaign,4.08.2019,1940,72878.0,61235.0,3065.0,1042.0,982.0,1183.0,340.0
4,Control Campaign,5.08.2019,1835,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
df_control.isnull().sum()

Campaign Name        0
Date                 0
Amount Spent         0
Num Impressions      1
Reach                1
Num Clicks           1
Num Searches         1
Num Views            1
Num Added to cart    1
Num Purchases        1
dtype: int64

In [52]:
df_control.describe()

,Amount Spent,Num Impressions,Reach,Num Clicks,Num Searches,Num Views,Num Added to cart,Num Purchases
count,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000
mean,2288.433333,109559.758621,88844.931034,5320.793103,2221.310345,1943.793103,1300.000000,522.793103
std,367.334451,21311.695472,21452.627592,1726.803732,851.025795,764.021907,400.371207,181.810508
min,1757.000000,71274.000000,42859.000000,2277.000000,1001.000000,848.000000,442.000000,222.000000
25%,1945.500000,95191.250000,75300.250000,4122.250000,1629.250000,1249.000000,942.500000,375.500000
50%,2299.500000,112368.000000,91418.000000,5272.396552,2340.000000,1979.500000,1319.500000,506.000000
75%,2532.000000,121259.000000,101958.750000,6609.500000,2655.000000,2360.500000,1638.000000,663.250000
max,3083.000000,145248.000000,127852.000000,8137.000000,4891.000000,4219.000000,1913.000000,800.000000


In [35]:
col_to_fill = ['Num Impressions', 'Reach',
    'Num Clicks', 'Num Searches', 'Num Views',
    'Num Added to cart', 'Num Purchases']

for col in col_to_fill:
  mean_value = df_control[col].mean()
  df_control[col].fillna(mean_value, inplace=True)

df_control.isnull().sum()

Campaign Name        0
Date                 0
Amount Spent         0
Num Impressions      0
Reach                0
Num Clicks           0
Num Searches         0
Num Views            0
Num Added to cart    0
Num Purchases        0
dtype: int64

### Test Base

In [36]:
df_test = pd.read_csv('test_group.csv', sep=';')
df_test

,Campaign Name,Date,Spend [USD],# of Impressions,Reach,# of Website Clicks,# of Searches,# of View Content,# of Add to Cart,# of Purchase
0,Test Campaign,1.08.2019,3008,39550,35820,3038,1946,1069,894,255
1,Test Campaign,2.08.2019,2542,100719,91236,4657,2359,1548,879,677
2,Test Campaign,3.08.2019,2365,70263,45198,7885,2572,2367,1268,578
3,Test Campaign,4.08.2019,2710,78451,25937,4216,2216,1437,566,340
4,Test Campaign,5.08.2019,2297,114295,95138,5863,2106,858,956,768
5,Test Campaign,6.08.2019,2458,42684,31489,7488,1854,1073,882,488
6,Test Campaign,7.08.2019,2838,53986,42148,4221,2733,2182,1301,890
7,Test Campaign,8.08.2019,2916,33669,20149,7184,2867,2194,1240,431
8,Test Campaign,9.08.2019,2652,45511,31598,8259,2899,2761,1200,845
9,Test Campaign,10.08.2019,2790,95054,79632,8125,2312,1804,424,275


In [37]:
df_test.columns = new_column_names

df_test.head()

,Campaign Name,Date,Amount Spent,Num Impressions,Reach,Num Clicks,Num Searches,Num Views,Num Added to cart,Num Purchases
0,Test Campaign,1.08.2019,3008,39550,35820,3038,1946,1069,894,255
1,Test Campaign,2.08.2019,2542,100719,91236,4657,2359,1548,879,677
2,Test Campaign,3.08.2019,2365,70263,45198,7885,2572,2367,1268,578
3,Test Campaign,4.08.2019,2710,78451,25937,4216,2216,1437,566,340
4,Test Campaign,5.08.2019,2297,114295,95138,5863,2106,858,956,768


In [38]:
df_test.isnull().sum()

Campaign Name        0
Date                 0
Amount Spent         0
Num Impressions      0
Reach                0
Num Clicks           0
Num Searches         0
Num Views            0
Num Added to cart    0
Num Purchases        0
dtype: int64

In [53]:
df_test.describe()

,Amount Spent,Num Impressions,Reach,Num Clicks,Num Searches,Num Views,Num Added to cart,Num Purchases
count,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000
mean,2563.066667,74584.800000,53491.566667,6032.333333,2418.966667,1858.000000,881.533333,521.233333
std,348.687681,32121.377422,28795.775752,1708.567263,388.742312,597.654669,347.584248,211.047745
min,1968.000000,22521.000000,10598.000000,3038.000000,1854.000000,858.000000,278.000000,238.000000
25%,2324.500000,47541.250000,31516.250000,4407.000000,2043.000000,1320.000000,582.500000,298.000000
50%,2584.000000,68853.500000,44219.500000,6242.500000,2395.500000,1881.000000,974.000000,500.000000
75%,2836.250000,99500.000000,78778.750000,7604.750000,2801.250000,2412.000000,1148.500000,701.000000
max,3112.000000,133771.000000,109834.000000,8264.000000,2978.000000,2801.000000,1391.000000,890.000000


### Comparison

In [39]:
df_control.describe()

,Amount Spent,Num Impressions,Reach,Num Clicks,Num Searches,Num Views,Num Added to cart,Num Purchases
count,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000
mean,2288.433333,109559.758621,88844.931034,5320.793103,2221.310345,1943.793103,1300.000000,522.793103
std,367.334451,21311.695472,21452.627592,1726.803732,851.025795,764.021907,400.371207,181.810508
min,1757.000000,71274.000000,42859.000000,2277.000000,1001.000000,848.000000,442.000000,222.000000
25%,1945.500000,95191.250000,75300.250000,4122.250000,1629.250000,1249.000000,942.500000,375.500000
50%,2299.500000,112368.000000,91418.000000,5272.396552,2340.000000,1979.500000,1319.500000,506.000000
75%,2532.000000,121259.000000,101958.750000,6609.500000,2655.000000,2360.500000,1638.000000,663.250000
max,3083.000000,145248.000000,127852.000000,8137.000000,4891.000000,4219.000000,1913.000000,800.000000


In [40]:
df_test.describe()

,Amount Spent,Num Impressions,Reach,Num Clicks,Num Searches,Num Views,Num Added to cart,Num Purchases
count,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000
mean,2563.066667,74584.800000,53491.566667,6032.333333,2418.966667,1858.000000,881.533333,521.233333
std,348.687681,32121.377422,28795.775752,1708.567263,388.742312,597.654669,347.584248,211.047745
min,1968.000000,22521.000000,10598.000000,3038.000000,1854.000000,858.000000,278.000000,238.000000
25%,2324.500000,47541.250000,31516.250000,4407.000000,2043.000000,1320.000000,582.500000,298.000000
50%,2584.000000,68853.500000,44219.500000,6242.500000,2395.500000,1881.000000,974.000000,500.000000
75%,2836.250000,99500.000000,78778.750000,7604.750000,2801.250000,2412.000000,1148.500000,701.000000
max,3112.000000,133771.000000,109834.000000,8264.000000,2978.000000,2801.000000,1391.000000,890.000000


# 2. Statistical Tests

### Shapiro-Wilk Test

- This is a statistical test used to check whether the given data sample of data comes from a normal distribution population.

- The null hypothesis (H0) is assuming that it is a normal distribution.

In [42]:
shapiro_list_results = []

test_stat_control, p_value_control = shapiro(df_control['Num Purchases'])

shapiro_list_results.append({'Group': 'Control', 'Test Statistic': test_stat_control, 'P-Value':p_value_control})

test_stat_test, p_value_test = shapiro(df_test['Num Purchases'])

shapiro_list_results.append({'Group': 'Test', 'Test Statistic': test_stat_test, 'P-Value':p_value_test})

df_shapiro_list_results = pd.DataFrame(shapiro_list_results)
df_shapiro_list_results

,Group,Test Statistic,P-Value
0,Control,0.943273,0.111445
1,Test,0.918189,0.024078



Assuming that:
- We have an alpha of 0.05
- p-value <= alpha (reject H0)
- p-value > alpha (fail to reject H0)

By the results above:
- Control: we failed to reject the H0 hypothesis, which could mean that it is a normal distribution.
- Test: we rejected the H0 hypothesis of the normal distribution. So Test is not normally distributed.

### T-Test

- This statistical test compares two samples to see if they are significantly different from each other
- It measures the difference in means relative to the variation of the data
- The null hypothesis means that there is no difference between the means of these 2 groups.


In [43]:
t_stat, p_value = ttest_ind(df_control['Num Purchases'], df_test['Num Purchases'])

t_stat, p_value

(0.03066909523750146, 0.9756387309702421)

Also assuming an alpha = 0.05

By the results above:

Since p-value > alpha, we failed to reject the null hypothesis, so there is no evidence for null hypothesis. Maybe there is no difference between these two samples.

### T-Test with new metrics

In [44]:
df_control['Type Data'] = 'Control'
df_control

,Campaign Name,Date,Amount Spent,Num Impressions,Reach,Num Clicks,Num Searches,Num Views,Num Added to cart,Num Purchases,Type Data
0,Control Campaign,1.08.2019,2280,82702.000000,56930.000000,7016.000000,2290.000000,2159.000000,1819.0,618.000000,Control
1,Control Campaign,2.08.2019,1757,121040.000000,102513.000000,8110.000000,2033.000000,1841.000000,1219.0,511.000000,Control
2,Control Campaign,3.08.2019,2343,131711.000000,110862.000000,6508.000000,1737.000000,1549.000000,1134.0,372.000000,Control
3,Control Campaign,4.08.2019,1940,72878.000000,61235.000000,3065.000000,1042.000000,982.000000,1183.0,340.000000,Control
4,Control Campaign,5.08.2019,1835,109559.758621,88844.931034,5320.793103,2221.310345,1943.793103,1300.0,522.793103,Control
5,Control Campaign,6.08.2019,3083,109076.000000,87998.000000,4028.000000,1709.000000,1249.000000,784.0,764.000000,Control
6,Control Campaign,7.08.2019,2544,142123.000000,127852.000000,2640.000000,1388.000000,1106.000000,1166.0,499.000000,Control
7,Control Campaign,8.08.2019,1900,90939.000000,65217.000000,7260.000000,3047.000000,2746.000000,930.0,462.000000,Control
8,Control Campaign,9.08.2019,2813,121332.000000,94896.000000,6198.000000,2487.000000,2179.000000,645.0,501.000000,Control
9,Control Campaign,10.08.2019,2149,117624.000000,91257.000000,2277.000000,2475.000000,1984.000000,1629.0,734.000000,Control


In [45]:
df_test['Type Data'] = 'Test'
df_test

,Campaign Name,Date,Amount Spent,Num Impressions,Reach,Num Clicks,Num Searches,Num Views,Num Added to cart,Num Purchases,Type Data
0,Test Campaign,1.08.2019,3008,39550,35820,3038,1946,1069,894,255,Test
1,Test Campaign,2.08.2019,2542,100719,91236,4657,2359,1548,879,677,Test
2,Test Campaign,3.08.2019,2365,70263,45198,7885,2572,2367,1268,578,Test
3,Test Campaign,4.08.2019,2710,78451,25937,4216,2216,1437,566,340,Test
4,Test Campaign,5.08.2019,2297,114295,95138,5863,2106,858,956,768,Test
5,Test Campaign,6.08.2019,2458,42684,31489,7488,1854,1073,882,488,Test
6,Test Campaign,7.08.2019,2838,53986,42148,4221,2733,2182,1301,890,Test
7,Test Campaign,8.08.2019,2916,33669,20149,7184,2867,2194,1240,431,Test
8,Test Campaign,9.08.2019,2652,45511,31598,8259,2899,2761,1200,845,Test
9,Test Campaign,10.08.2019,2790,95054,79632,8125,2312,1804,424,275,Test


In [48]:
df_merge = pd.concat([df_control, df_test], ignore_index=True)
df_merge.head()

,Campaign Name,Date,Amount Spent,Num Impressions,Reach,Num Clicks,Num Searches,Num Views,Num Added to cart,Num Purchases,Type Data
0,Control Campaign,1.08.2019,2280,82702.000000,56930.000000,7016.000000,2290.000000,2159.000000,1819.0,618.000000,Control
1,Control Campaign,2.08.2019,1757,121040.000000,102513.000000,8110.000000,2033.000000,1841.000000,1219.0,511.000000,Control
2,Control Campaign,3.08.2019,2343,131711.000000,110862.000000,6508.000000,1737.000000,1549.000000,1134.0,372.000000,Control
3,Control Campaign,4.08.2019,1940,72878.000000,61235.000000,3065.000000,1042.000000,982.000000,1183.0,340.000000,Control
4,Control Campaign,5.08.2019,1835,109559.758621,88844.931034,5320.793103,2221.310345,1943.793103,1300.0,522.793103,Control


In [49]:
# Creating new metrics:
# Click-through Rate: Number of website clicks received through the ads/ Impressions
df_merge['CTR'] = (df_merge['Num Clicks'] / df_merge['Num Impressions'])*100
# Conversion Rate: Users who purchased / Users who clicked
df_merge['Conversion Rate'] = (df_merge['Num Purchases']/df_merge['Num Clicks']) * 100
# Cost per Click: amount spent on that campaign / Number of clicks
df_merge['CPC'] = df_merge['Amount Spent']/df_merge['Num Clicks']
# ROI: net revenue/ investment
df_merge['ROI'] = ((df_merge['Num Purchases'] - df_merge['Amount Spent'])/df_merge['Amount Spent'])*100
df_merge.head()


,Campaign Name,Date,Amount Spent,Num Impressions,Reach,Num Clicks,Num Searches,Num Views,Num Added to cart,Num Purchases,Type Data,CTR,Conversion Rate,CPC,ROI
0,Control Campaign,1.08.2019,2280,82702.000000,56930.000000,7016.000000,2290.000000,2159.000000,1819.0,618.000000,Control,8.483471,8.808438,0.324971,-72.894737
1,Control Campaign,2.08.2019,1757,121040.000000,102513.000000,8110.000000,2033.000000,1841.000000,1219.0,511.000000,Control,6.700264,6.300863,0.216646,-70.916335
2,Control Campaign,3.08.2019,2343,131711.000000,110862.000000,6508.000000,1737.000000,1549.000000,1134.0,372.000000,Control,4.941121,5.716042,0.360018,-84.122919
3,Control Campaign,4.08.2019,1940,72878.000000,61235.000000,3065.000000,1042.000000,982.000000,1183.0,340.000000,Control,4.205659,11.092985,0.632953,-82.474227
4,Control Campaign,5.08.2019,1835,109559.758621,88844.931034,5320.793103,2221.310345,1943.793103,1300.0,522.793103,Control,4.856521,9.825473,0.344873,-71.509913


In [51]:
metrics_to_analyse = ['CTR', 'Conversion Rate', 'CPC', 'ROI']

metrics_to_analyse_results = []

for metric in metrics_to_analyse:
  t_stat_new, p_value_new = ttest_ind(df_merge[df_merge['Type Data']=='Control'][metric], df_merge[df_merge['Type Data']=='Test'][metric])

  metric_results = pd.DataFrame({'Metric': [metric],
                                   'T-Statistic': [t_stat_new],
                                   'P-Value': [p_value_new]})
  metrics_to_analyse_results.append(metric_results)

t_test_results = pd.concat(metrics_to_analyse_results, ignore_index=True)

t_test_results

,Metric,T-Statistic,P-Value
0,CTR,-3.996250,0.000184
1,Conversion Rate,1.488079,0.142147
2,CPC,0.410837,0.682706
3,ROI,1.183435,0.241465


Through Conversion Rate, CPC and ROI:
- p-value > alpha: We failed to reject the null hypothesis, so maybe there is no difference between these 2 samples using these metrics.

Through CTR:
- p-value < alpha: We rejected the null hypotheses. So there is difference between these 2 samples using CTR.

# References

In [ ]:
# https://www.kaggle.com/code/yaminh/data-driven-ab-testing-for-marketing-excellence